In [ ]:
import pandas as pd
from tqdm import tqdm
import os, shutil, glob
import subprocess
from subprocess import Popen, PIPE
import sys
import re
from bs4 import BeautifulSoup
import javalang

In [ ]:
def findAbsolutePathMyMachine(path, classes):
    for jClass in classes:
        #print(jClass)
        if path in jClass:
            return jClass
        
base_path = '/Copilot-Robustness/Ready/'
generated_instances_path = '/Dataset/CopilotInstances/FullContext/'
df = pd.read_csv('/manual.csv',index_col="index")

In [ ]:
pattern = re.compile(r"(?:(?:public|private|protected|static|final|native|synchronized|abstract|transient)+\s+)+[$_\w<>\[\]\s]*\s+[\$_\w]+\([^\)]*\)?\s*\{?[^\}]*\}?")

def extractGeneratedMethod(supposedToBeMethod, startingLine, endingLine):
    

    spanToLine = startingLine
    flagQuote = False
    flagValid = False
    counter = 0
    
    fromTo = '\n'.join(supposedToBeMethod.splitlines()[startingLine+1:])
    
    try:
        nextSignature = pattern.findall(fromTo)[0].split('\n')[0]
    except Exception:
        # if fails, then we reached the EOF
        nextSignature = ""
    
    span = endingLine - startingLine
    
    for line in supposedToBeMethod.splitlines()[startingLine:]:
                
        for character in line:
            
            if character == '"' and not flagQuote:
                flagQuote = True
            
            else:
                if character == '"' and flagQuote:
                    flagQuote = False
                
            if character == '{' and not flagQuote:
                counter += 1
            
            if character == '}' and not flagQuote:
                #print('hit')
                counter -= 1

        
       
        if counter == 0 and ''.join(line.strip().split()) != ''.join(nextSignature.strip().split()):
            flagValid = True
            break

        if ''.join(line.strip().split()) == ''.join(nextSignature.strip().split()):
            flagValid = False
            break
            
        else:
            spanToLine += 1
    
    
    flagEmpty = True
    for line in supposedToBeMethod.splitlines()[startingLine+1:spanToLine]:
        if line.strip() != '':
            flagEmpty = False
            break
    
    if flagEmpty:
        return -2
    
    if flagValid:
        return spanToLine+1
    
    if not flagValid:
        return -1
    
    return -1
    
    #return -1
    
def sanitySaveCheck(javaClass, targetString):
    if targetString in javaClass:
        return False
    else:
        return True

In [ ]:
df['generatedResultOriginal'] = ["" for item in range(0,1006)]
df['generatedResultP0'] = ["" for item in range(0,1006)]
df['generatedResultP1'] = ["" for item in range(0,1006)]
df['generatedResultP2'] = ["" for item in range(0,1006)]

In [ ]:
resultOriginalList = []
resultPerturbated_0_List = []
resultPerturbated_1_List = []
resultPerturbated_2_List = []

for (idx,row) in tqdm(df.iterrows()):
    #print(idx)
    
    indexFolder = idx #row['index']
    startingMethodLine = int(row['spanMethod'].split('-')[0])+1
    endingMethodLine = int(row['spanMethod'].split('-')[1])
    
    
    ######################################################################################
       
    #try:
    path_to_original = generated_instances_path + str(indexFolder) + '/' + 'Original.java'
    with open(path_to_original) as f:
        originalJava = f.read()

    path_to_original_result = generated_instances_path + str(indexFolder) + '/' + 'ResultOriginal.java'
    with open(path_to_original_result) as f:
        originalJavaResult = f.read()

    if(not sanitySaveCheck(originalJavaResult, 'ResultOriginal')):
        print(indexFolder)


    endingLine = extractGeneratedMethod(originalJavaResult, startingMethodLine, endingMethodLine)
    if endingLine == -1:
        resultOriginalList.append("Not Valid")
        df.loc[indexFolder, 'generatedResultOriginal'] = "Not Valid"

    elif endingLine == -2:
        df.loc[indexFolder, 'generatedResultOriginal'] = "Empty Method"

    else:
        result = '\n'.join(originalJavaResult.splitlines()[startingMethodLine:endingLine])
        resultOriginalList.append(result)
        df.loc[indexFolder, 'generatedResultOriginal'] = result



    ######################################################################################

    path_to_p0 = generated_instances_path + str(indexFolder) + '/' + 'Perturbated_0.java'
    with open(path_to_p0) as f:
        perturbated_0_Java = f.read()

    path_to_p0_result = generated_instances_path + str(indexFolder) + '/' + 'ResultPerturbated0.java'
    with open(path_to_p0_result) as f:
        perturbated_0_JavaResult = f.read()


    if(not sanitySaveCheck(perturbated_0_JavaResult, 'ResultPerturbated0')):
        print(indexFolder)


    endingLine = extractGeneratedMethod(perturbated_0_JavaResult, startingMethodLine, endingMethodLine)
    #print('--> ',endingLine)
    if endingLine == -1:
        resultPerturbated_0_List.append("Not Valid")
        df.loc[indexFolder, 'generatedResultP0'] = "Not Valid"

    elif endingLine == -2:
        df.loc[indexFolder, 'generatedResultP0'] = "Empty Method"

    else:
        resultPerturbated_0_List.append(perturbated_0_JavaResult.splitlines()[startingMethodLine:endingLine])
        result = '\n'.join(perturbated_0_JavaResult.splitlines()[startingMethodLine:endingLine])
        resultPerturbated_0_List.append(result)
        df.loc[indexFolder, 'generatedResultP0'] = result



    ######################################################################################

    path_to_p1 = generated_instances_path + str(indexFolder) + '/' + 'Perturbated_1.java'
    with open(path_to_p1) as f:
        perturbated_1_Java = f.read()

    path_to_p1_result = generated_instances_path + str(indexFolder) + '/' + 'ResultPerturbated1.java'
    with open(path_to_p1_result) as f:
        perturbated_1_JavaResult = f.read()

    if(not sanitySaveCheck(perturbated_1_JavaResult, 'ResultPerturbated1')):
        print(indexFolder)

    endingLine = extractGeneratedMethod(perturbated_1_JavaResult, startingMethodLine, endingMethodLine)
    #print('--> ',endingLine)
    if endingLine == -1:
        #print(' {} ----- {} '.format(indexFolder, startingMethodLine))
        resultPerturbated_1_List.append("Not Valid")
        df.loc[indexFolder, 'generatedResultP1'] = "Not Valid"

    elif endingLine == -2:
        df.loc[indexFolder, 'generatedResultP1'] = "Empty Method"    

    else:
        result = '\n'.join(perturbated_1_JavaResult.splitlines()[startingMethodLine:endingLine])
        resultPerturbated_1_List.append(result)
        df.loc[indexFolder, 'generatedResultP1'] = result


    ######################################################################################

    path_to_p2 = generated_instances_path + str(indexFolder) + '/' + 'Perturbated_2.java'
    with open(path_to_p2) as f:
        perturbated_2_Java = f.read()

    path_to_p2_result = generated_instances_path + str(indexFolder) + '/' + 'ResultPerturbated2.java'
    with open(path_to_p2_result) as f:
        perturbated_2_JavaResult = f.read()


    if(not sanitySaveCheck(perturbated_2_JavaResult, 'ResultPerturbated2')):
        print(indexFolder)

    endingLine = extractGeneratedMethod(perturbated_2_JavaResult, startingMethodLine, endingMethodLine)
    #print('--> ',endingLine)
    if endingLine == -1:
        resultPerturbated_2_List.append("Not Valid")
        df.loc[indexFolder, 'generatedResultP2'] = "Not Valid"

    elif endingLine == -2:
        df.loc[indexFolder, 'generatedResultP2'] = "Empty Method"

    else:
        result = '\n'.join(perturbated_2_JavaResult.splitlines()[startingMethodLine:endingLine])
        resultPerturbated_2_List.append(result)
        df.loc[indexFolder, 'generatedResultP2'] = result


        ######################################################################################




In [ ]:
df.to_csv('/analyze.csv')

In [ ]:
# def flatten(multiline):
#     lst = multiline.split('\n')
#     flat = ''
#     for line in lst:
#         flat += line.replace(' ', '')+' '
#     return ''.join(flat.split())

# perfectOriginal = 0
# perfectP0 = 0
# perfectP1 = 0
# perfectP2 = 0

# for (idx,row) in tqdm(df.iterrows()):
#     body = flatten(row['body'])
#     resultOriginal = flatten(row['generatedResultOriginal'])
#     resultP0 = flatten(row['generatedResultP0'])
#     resultP1 = flatten(row['generatedResultP1'])
#     resultP2 = flatten(row['generatedResultP2'])
    
#     if resultOriginal==body:
#         perfectOriginal += 1
    
#     if resultP0==body:
#         perfectP0 += 1
        
#     if resultP1==body:
#         perfectP1 += 1
        
#     if resultP1==body:
#         perfectP2 += 1

# print("Original Instance PP {}/{} = {}".format(perfectOriginal,len(df), ( (perfectOriginal/len(df)) * 100 ) ))
# print("Perturbated@0 Instance PP {}/{} = {}".format(perfectP0,len(df),  ( (perfectP0/len(df)) * 100 ) ))
# print("Perturbated@1 Instance PP {}/{} = {}".format(perfectP1,len(df),  ( (perfectP1/len(df)) * 100 ) ))
# print("Perturbated@2 Instance PP {}/{} = {}".format(perfectP2,len(df),  ( (perfectP2/len(df)) * 100 ) ))